### **Title**: Finding Your Stuff in Flywheel with the Flywheel SDK
### **Date**:  May-2020  
### **Description**:  
* This notebook is intended to accompany a 2-part Flywheel webinar series on finding and accessing containers in Flywheel using the Flywheel Python SDK.
* The code herein will not change any metadata in the Flywheel SDK - only "read" actions will be performed.
* The following are required to use this Notebook:
    1. Read access to a Flywheel project that contains a DICOM acquisition file. 
    1. A flywheel API key
    1. An environment in which to execute the notebook (Binder/Colab/JupyterLab/etc)
    

# Part 1

## Install and import dependencies

In [ ]:
# Install specific packages required for this notebook
!pip install flywheel-sdk

In [ ]:
# Import packages
from getpass import getpass
import logging
import os
import pprint
import re
from IPython.display import Markdown as md

import flywheel

In [ ]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

## Flywheel Client Initialization with an API key

Any time you use the flywheel SDK, you will need to initialize a client with your API key. 

To find your API key, first click on the circle in the top right corner of Flywheel, then click on `Profile` as shown below: 

![profile_location](assets/profile_location.png)

On your profile page, your API key will be located under the `Your API Key` as below:

![api_key_location](assets/api_key_location.png)

Copy this key, run the cell below, paste the key into the `Enter API_KEY here:` box and press return.

While you can initialize your client as `fw` with `fw = flywheel.Client('<your api key>')`, it is best to obscure credentials from your code, especially when sharing with others. 

**CLEAR YOUR OUTPUTS BEFORE SHARING THIS NOTEBOOK**, as your API key can be retrieved from the client. 

In [ ]:
# Password widget/prompt (good security practice)
API_KEY = getpass('Enter API_KEY here: ')
# Initialize the client
fw = flywheel.Client(API_KEY or os.environ.get('FW_KEY'))
# Clean up the API_KEY
del API_KEY
# Log information about site/user
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

## Using `lookup` to find items

If you know the group **id** and container(project/subject/session/acquisition) labels, fw.lookup() can be used to return container and file objects via the SDK. In order to use lookup, we need to know how to format the **resolver path**, the Flywheel equivalent to a local file path. Let's look at how to locate the components of a resolver path and some examples!

### locating group id and project label

It is important to distinguish between group ID and group label. This is illustrated by the image below:
![group_id_label](assets/group_id_label.png)

In the above image:
* group id is `scien`
* group label is `Scientific Solutions`
* project label is `MIDAS_veggies`

To retrieve the project via lookup, the following command would be used:

```python
fw.lookup('scien/MIDAS_veggies')
```

## locating subject, session, acquisition labels, file name

![labels_file_name](assets/labels_file_name.png)

To retrieve subject `dragon_fruit` via lookup:

```python
fw.lookup('scien/MIDAS_veggies/dragon_fruit')
```

To retrieve subject `dragon_fruit`'s `session_1` via lookup:

```python
fw.lookup('scien/MIDAS_veggies/dragon_fruit/session_1')
```

To retrieve `acq1` from `dragon_fruit`'s `session_1` via lookup:

```python
fw.lookup('scien/MIDAS_veggies/dragon_fruit/session_1/acq1')
```

To retrieve file `IM_0001` from `acq1` we need to add `files` to the path as below:

```python
fw.lookup('scien/MIDAS_veggies/dragon_fruit/session_1/acq1/files/IM_0001')
```

Files for any container can be retrieved by adding `/files/<your file name here>` to the container path.

In the code block below, substitute the group id, container labels, and file name variables with those from your data.

In [ ]:
# Set group id and labels (add your labels within the single quotes)
group_id = 'scien'
project_label = 'MIDAS_veggies'
subject_label = 'dragon_fruit'
session_label = 'session_1'
acquisition_label = 'acq1'
file_name = 'IM_0001'


def lookup_demo(fw_client, resolver_path, log):
    """
    lookup container at resolver path using fw_client
    Args:
        fw_client (flywheel.Client): an instance of the flywheel client
        resolver_path (str): the resolver path to a flywheel container
        log (logging.Logger): the logger to use
    Returns:
        a flywheel container object (the result of fw_client.lookup(lookup_str)
    """
    log.info(f'resolver_path:\t {resolver_path}')
    try:
        container = fw_client.lookup(resolver_path)
        label = container.get('label') or container.get('name')
        print_str = (f'{label} is a {container.container_type} with id {container.id}\n')
        log.info(print_str)
        return container
    
    except flywheel.rest.ApiException as e:
        log.error(
            'An exception was raised when trying to resolve the container at resolver path %s', 
            resolver_path,
            exc_info=True
        )
        return None

# Lookup group
resolver_path = group_id
group = lookup_demo(fw, resolver_path, log)

# Lookup project
resolver_path = '/'.join([resolver_path, project_label])
project = lookup_demo(fw, resolver_path, log)

# Lookup subject
resolver_path = '/'.join([resolver_path, subject_label])
subject = lookup_demo(fw, resolver_path, log)

# Lookup session
resolver_path = '/'.join([resolver_path, session_label])
session = lookup_demo(fw, resolver_path, log)

# Lookup acquisition
resolver_path = '/'.join([resolver_path, acquisition_label])
acquisition = lookup_demo(fw, resolver_path, log)


# Lookup acquisition file 
resolver_path = '/'.join([resolver_path, 'files', file_name])
acquisition_file = lookup_demo(fw, resolver_path, log)

# Lookup non-existent resolver - we expect a traceback here since it does not exist
resolver_path = 'group/does/not/exist'
none_result = lookup_demo(fw, resolver_path, log)

# Using `get` to retrieve containers

`fw.lookup()` works well when the resolver path is uniquely descriptive for the file/container in question. However, if you have multiple sessions/acquisitions/files with the same labels/resolver paths. `get` is typically faster than `lookup`, especially for large projects. `get_<container type>` (for example `get_session`) functions are typically faster than `get` and can be used when you know the container type of the container you are trying to retrieve.


## Locating container IDs to pass to `get` methods

In order to retrieve a container via get, we need to obtain its **id**, a value that is assigned by Flywheel to uniquely identify containers

### project and session IDs

Within sessions view, the url will contain project ID to the right of `projects/` and session ID to the right of `sessions/`
![project_session_id](assets/project_session_id.png)

In this example, we could use either of the following to retrieve the project:

``` python
fw.get('5e90d7a5a3803400a8e63b13')
fw.get_project('5e90d7a5a3803400a8e63b13')
```

And to retrieve the session:

``` python
fw.get('5e948782a38034010ce63ac7')
fw.get_session('5e948782a38034010ce63ac7')
```

### acquisition ID

1. Right-click + "Inspect" - the inspection window displayed in 3 will appear

    ![inspect](assets/inspect.png)
    
2. Click "kebab" menu to the right of acquisition label, then select information:

    ![kebab_info](assets/kebab_info.png)
    
3. Select "Network" tab within inspection window, locate acquisition ID:
    
    ![acquisition_id_network](assets/acquisition_id_network.png)
    
To retrieve the acquisition in this example, we would use the following:

```python
fw.get('5e948799a380340100e63acb')
fw.get_acquisition('5e948799a380340100e63acb')
```
    
### subject ID

Click on the person icon to enter subjects view, then click on the row with the subject label. The url will now have `subjects/`, followed by the subject ID:

![subject_id](assets/subject_id.png)

To retrieve the subject in this example, we would use the following:

```python
fw.get('5e948782a38034010ce63ac6')
fw.get_subject('5e948782a38034010ce63ac6')
```

### getting a container file

Files cannot be obtained via `get` methods. Instead, you must get the parent container and use the `get_file` method to return the file object.

To illustrate, the following code would be used to retrieve acq1's IM_0001 file:

```python
acq = fw.get_acquisition('5e948799a380340100e63acb')

acq.get_file('IM_0001')
```

## Additional tips for exploring the SDK
* When examining a python object, it is useful to examine its attributes with `dir`

    ```python
    acq = fw.get_acquisition('5e948799a380340100e63acb')
    print(dir(acq))
    ```
    
* Notebooks will display information about a function/method when followed by a `?` `dir?`, for example
    * You can also print the docstring in other IDEs with `print(dir.__doc__)`

In [ ]:
# Add your container id between the single quotes
container_id = '5e948799a380340100e63acb'
container = fw.get(container_id)
print(f'{container.label} is a {container.container_type} with id: {container.id}\n')


# What methods/attributes are available?
print('dir(container):\n')
pprint.pprint(dir(container))
print('\n')

# Print out the container JSON
pprint.pprint(container.to_dict())



## Example: generate the url for a session, subject, or project

In [ ]:
from IPython.display import display, HTML

def get_uri_prefix(client_config_site_api_url):
    """
    Removes /api and port (i.e. :443) from client_config_site_api_url
    Args:
        client_config_site_api_url (str): the value for client.get_config().site.api_url

    Returns:
        str: the uri without /api or port information
    """
    remove_regex = r'(:[\d]+)?/api'
    return_prefix = re.sub(remove_regex, '', client_config_site_api_url)
    return return_prefix

def create_container_link(fw_client, container):
    api_url = fw_client.get_config().site.api_url
    prefix = get_uri_prefix(api_url)
    if container.container_type == 'project':
        return_url = '/'.join([prefix, '#', 'projects', container.id])
    elif container.container_type == 'session':
        return_url = '/'.join([prefix, '#', 'projects', container.project, 'sessions', container.id])
    elif container.container_type == 'subject':
        return_url = '/'.join([prefix, '#', 'projects', container.project, 'subjects', container.id])
    else:
        log.error('Link creation is not supported for %s containers', container.container_type)
        return_url = None
        
    return return_url

session_link = create_container_link(fw, session)
link_t = '<a href=\'{}\'> Session Link </a>'
html = HTML(link_t.format(session_link))
display(html)

subject_link = create_container_link(fw, subject)
link_t = '<a href=\'{}\'> Subject Link </a>'
html = HTML(link_t.format(subject_link))
display(html)

In [ ]:
api_url = fw.get_config().site.api_url
prefix = get_uri_prefix(api_url)
advanced_search_url = '/'.join([prefix, '#', 'search', 'advanced'])
link_t = '<a href=\'{}\'> Click to navigate to {} </a>'
html = HTML(link_t.format(advanced_search_url, advanced_search_url))
display(html)

# Constructing advanced search queries

images to illustrate using advanced search (live demo? video?)

term interface vs typing

view all data for admins

To run the query formatted with the UI tool:

```python
query = 'file.type = dicom AND subject.label = dragon_fruit'
fw.search({'return_type': 'acquisition', 'structured_query': query})
```

# Part 2